In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# X_train, Y_train = load_cifar_10_data_batch(path + '/cifar-10-batches-py/data_batch_1')
# X_test, Y_test = load_cifar_10_data_batch

In [3]:
with np.load('Buildingtop.npz') as Buildingtop:
    train_x = Buildingtop['train_x']
    train_y = Buildingtop['train_Y']
    test_x = Buildingtop['test_x']
    test_y = Buildingtop['test_y']
with np.load('Treetop.npz') as Treetop:
    train_x = np.vstack((train_x,Treetop['train_x']))
    train_y = np.hstack((train_y,Treetop['train_Y']))
    test_x = np.vstack((test_x,Treetop['test_x']))
    test_y = np.hstack((test_y,Treetop['test_y']))
with np.load('BigBuilding.npz') as BigBuilding:
    train_x = np.vstack((train_x,BigBuilding['train_x']))
    train_y = np.hstack((train_y,BigBuilding['train_Y']))
    test_x = np.vstack((test_x,BigBuilding['test_x']))
    test_y = np.hstack((test_y,BigBuilding['test_y']))
with np.load('skycloudcontrast.npz') as skycloudcontrast:
    train_x = np.vstack((train_x,skycloudcontrast['train_x']))
    train_y = np.hstack((train_y,skycloudcontrast['train_Y']))
    test_x = np.vstack((test_x,skycloudcontrast['test_x']))
    test_y = np.hstack((test_y,skycloudcontrast['test_y']))

In [4]:
print(np.shape(train_x))
print(np.shape(train_y))
print(np.shape(test_x))
print(np.shape(test_y))

# train_y=train_y.reshape(200,1)
# test_y=test_y.reshape(20,1)
num_example=train_x.shape[0]
arr=np.arange(num_example)
np.random.shuffle(arr)
train_x=train_x[arr]
train_y=train_y[arr]
# train_y=train_y.reshape(200,)
# test_y=test_y.reshape(20,)
print(np.shape(train_x))
print(np.shape(train_y))

(400, 129792)
(400,)
(80, 129792)
(80,)
(400, 129792)
(400,)


In [5]:
X_train=train_x
Y_train=train_y
X_test=test_x
Y_test=test_y

In [6]:
print(train_y)

[0 3 2 1 0 0 1 3 0 0 2 3 1 1 0 0 0 3 3 0 2 2 3 3 2 1 2 0 2 1 0 2 2 1 3 1 0
 0 1 0 2 1 1 1 0 2 2 3 2 1 1 2 2 0 0 3 2 0 2 1 1 2 2 2 0 1 3 2 2 0 3 0 2 3
 1 0 2 3 3 2 0 0 0 2 3 3 2 1 3 3 1 2 3 2 3 0 2 3 3 2 3 0 2 2 1 2 2 2 0 3 1
 2 2 2 3 1 0 2 3 3 2 0 3 0 1 1 1 0 0 2 0 2 3 1 2 1 0 3 0 3 1 3 3 0 0 1 0 0
 0 0 0 1 1 2 2 2 2 0 0 1 1 0 2 3 0 2 1 3 1 1 3 1 1 0 0 1 0 1 2 1 2 3 1 1 1
 1 2 0 0 0 3 1 3 2 2 3 3 3 1 0 2 2 3 2 3 3 2 3 0 1 3 1 0 2 2 3 0 0 3 1 3 2
 2 0 3 2 1 1 3 1 3 3 0 1 2 1 3 0 0 3 1 3 1 2 1 3 3 1 0 0 3 0 2 1 2 3 0 2 1
 3 1 3 0 2 3 0 0 0 3 3 2 0 1 1 1 1 0 3 2 1 3 0 1 0 3 2 2 2 2 1 0 1 0 2 2 0
 3 2 3 1 1 3 0 1 1 0 2 1 0 0 2 0 1 1 3 0 3 1 3 0 1 2 2 1 3 2 2 3 0 3 3 0 3
 1 1 1 3 1 0 3 3 1 2 1 0 0 2 3 2 1 3 1 2 1 2 0 2 0 0 3 2 3 2 1 1 2 0 3 2 1
 0 2 3 3 2 3 3 1 3 0 1 2 0 1 0 2 3 3 0 1 1 3 3 2 1 3 1 0 0 3]


In [ ]:
# 2.构造网络所需的函数
# 2.1 参数w初始化, tf.random_normal为根据shape产生随机分布,均值标准差自定, shape=[height, width, channels, filters]
def weights(shape):
    return tf.Variable(tf.random_normal(shape=shape, mean=0, stddev=0.1))
# 2.2 参数b初始化, tf.constant为根据shape产生常量
def biases(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))
# 2.3 卷积函数, x=[m, height, width, channels], m为待训练集数量
def conv2d(x, W):
    return tf.nn.conv2d(input=x, filter=W, strides=[1,1,1,1], padding='SAME', use_cudnn_on_gpu=True)
# 2.4 池化函数, 池化其实也相当于一次卷积, 比较特殊的卷积
def max_pool(x):
    return tf.nn.max_pool(value=x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
# 2.5 计算准确率, 根据测试集
def compute_accuracy(x_test, y_test):
    global predict_function
    predict_test = sess.run(predict_function, feed_dict={xs:x_test})
    # 比较预测值与测试集最大值下标是否相同, 返回相同个数
    """
    =================================
    """
    y_test_one_hot = tf.one_hot(y_test, depth = 3)
    correct_num = tf.equal(tf.argmax(predict_test, 1), tf.argmax(y_test_one_hot, 1))
    # 计算均值即为精确率, 并转格式float32
    accuracy = tf.reduce_mean(tf.cast(correct_num, tf.float32))
    return sess.run(accuracy, feed_dict={xs:x_test, ys:y_test})
"""
=================
"""
# 3.定义输入, xs为训练集输入m*3072矩阵, ys为训练集labelsm*1矩阵
xs = tf.placeholder(tf.float32, [None, 416*104*3])/255
ys = tf.placeholder(tf.int32, [None])

# 4. 构造网络
# x0为输入m*32*32*3, 图像数量m, 大小32*32, channels为3
x0 = tf.transpose(tf.reshape(xs, [-1, 3, 104, 416]), perm=[0, 2, 3, 1])
# 4.1 构建卷积层, 卷积、池化、再卷积、再池化、最后拉伸
W_conv1 = weights([5, 5, 3, 30])
b_conv1 = biases([30])
a_conv1 = conv2d(x0, W_conv1) + b_conv1
z_conv1 = tf.nn.relu(a_conv1)
h_conv1 = max_pool(z_conv1)

W_conv2 = weights([5, 5, 30, 60])
b_conv2 = biases([60])
a_conv2 = conv2d(h_conv1, W_conv2) + b_conv2
z_conv2 = tf.nn.relu(a_conv2)
h_conv2 = max_pool(z_conv2)

# x_conv0为卷积池化后拉伸的输入m*(8*8*35=2240), 图像数量m, 输入为2240
"""
==============
"""
x_conv0 = tf.reshape(h_conv2, [-1, 104*26*60])
# 4.2 接入fully connected networks, 网络各层单元数为2240, 1024, 10
"""
=============
"""
W_fc1 = weights([104*26*60, 1024])
b_fc1 = biases([1024])
a_fc1 = tf.matmul(x_conv0, W_fc1) + b_fc1
z_fc1 = tf.nn.sigmoid(a_fc1)
"""
==========================
"""
W_fc2 = weights([1024, 3])
b_fc2 = biases([3])
a_fc2 = tf.matmul(z_fc1, W_fc2) + b_fc2
# z_fc2 = tf.nn.sigmoid(a_fc2)


# 4.3 预测函数与代价函数, 采用最简单的
predict_function = a_fc2
"""
=============================
"""
ys_one_hot = tf.one_hot(ys, depth = 3)
# cost_function = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(predict_function), reduction_indices=[1]))
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=ys_one_hot, logits=predict_function))

# 4.4 梯度下降, 使cost_function值最小
train_step = tf.train.AdamOptimizer(0.01).minimize(cost_function)
# train_step = tf.train.AdamOptimizer(1e-3).minimize(cost_function)

# 5. 训练并图像化表示
sess = tf.Session()
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

for i in range(20):
    for t in range(0, 400-10, 10):
        iter = t // 10
        xs_batch, ys_batch = X_train[t:t+10], Y_train[t:t+10]
        sess.run(train_step, feed_dict={xs:xs_batch, ys:ys_batch})
        if iter % 10== 0:
            cost = sess.run(cost_function, feed_dict={xs: xs_batch, ys: ys_batch})
            accuracy = compute_accuracy(X_test, Y_test)
            print("iters:%s, cost:%s, accuracy:%s" % (iter, cost, accuracy))
            print("Building top: ", compute_accuracy(X_test[0:20], Y_test[0:20]))
            print("tree top: ", compute_accuracy(X_test[20:40], Y_test[20:40]))
            print("big building: ", compute_accuracy(X_test[40:60], Y_test[40:60]))
            print("contrast: ", compute_accuracy(X_test[60:80], Y_test[60:80]))
# saver.save(sess, '/Users/wanglei/Downloads/test/model.ckpt')


"""
============================
"""
# for i in range(20):
#     xs_batch, ys_batch = X_train, Y_train
#     sess.run(train_step, feed_dict={xs:xs_batch, ys:ys_batch})
#     cost = sess.run(cost_function, feed_dict={xs: xs_batch, ys: ys_batch})
#     accuracy = compute_accuracy(X_test, Y_test)
#     print("iters:%s, cost:%s, accuracy:%s" % (i, cost, accuracy))

sess.close()


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

iters:0, cost:2.5543647, accuracy:0.5
Building top:  0.0
tree top:  1.0
big building:  0.0
contrast:  1.0
iters:10, cost:1.1634104, accuracy:0.5875
Building top:  0.35
tree top:  1.0
big building:  0.0
contrast:  1.0
iters:20, cost:0.20856228, accuracy:0.6125
Building top:  1.0
tree top:  0.1
big building:  0.85
contrast:  0.5
iters:30, cost:0.8783742, accuracy:0.5
Building top:  1.0
tree top:  0.15
big building:  0.45
contrast:  0.4
iters:0, cost:0.5068091, accuracy:0.6
Building top:  0.8
tree top:  0.3
big building:  0.4
contrast:  0.9
iters:10, cost:0.4039597, accuracy:0.6375
Building top:  0.95
tree top:  0.65
big building:  0.0
contrast:  0.95
iters:20, cost:0.13863334, accuracy:0.5875
Building top:  0.95
tree top:  0.0
big building:  0.9
contrast:  0.5
iters:30, cost:0.4804523, accuracy:0.58